In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=0adea99d4b338ecb84ac7f3d4f8539ea4cf5e14825e722ddab7da3ea1d7f1913
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.6MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F24 '''
def F24(X):
    f = bn.F24()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.02540893219954618 ,random_state=0 , l1_ratio=0.9393939393939394, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 40.098706 , C = 916.924678 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_241000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_24_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
        min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=925407, Mon Apr 13 20:10:26 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 2.183194320660534e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.0
    2     30 2.189779315874690e+03 1.0e+00 2.29e+00  2e+00  2e+00 0:00.0
    3     45 2.181870242252017e+03 1.1e+00 2.25e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2258.10596692] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2203.16075759] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 9.169269478110118e+02 1.5e+00 4.50e-01  4e-01  5e-01 0:01.7
  200   3000 7.586160471673143e+02 1.5e+00 3.01e-02  3e-02  3e-02 0:03.3
  300   4500 7.580806400895685e+02 1.5e+00 1.48e-03  1e-03  1e-03 0:04.9
  400   6000 7.580793731946915e+02 1.6e+00 6.57e-05  5e-05  6e-05 0:06.6
  500   7500 7.580793711984099e+02 1.6e+00 4.12e-06  3e-06  4e-06 0:08.2
  600   9000 7.580793711918975e+02 1.6e+00 2.04e-07  2e-07  2e-07 0:09.9
  700  10500 7.580793711918745e+02 1.7e+00 2.69e-08  2e-08  2e-08 0:11.5
  728  10920 7.580793711918745e+02 1.7e+00 1.84e-08  1e-08  2e-08 0:12.0
termination on tolfun=1e-11 (Mon Apr 13 20:10:40 2020)
final/bestever f-value = 7.580794e+02 7.580794e+02
incumbent solution: [-0.58279107  1.65000181  1.00086079 -0.70876565  1.02024492 -1.09544776
 -0.13443935 -0.82787474 ...]
std deviations: [1.43072853e-08 1.38905258e-08 1.46324679e-08 1.45557193e-08
 1.44230686e-08 1.51168911e-08 1.48149820e-08 1.48605354e-08 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimen

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2260.23872349] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=2260.238723, sigma=2.38e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=2260.238723, sigma=2.38e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -1.476726920658210e+03 1.5e+00 3.24e-01  3e-01  3e-01 0:02.0
  200   3000 -1.587014856493383e+03 1.6e+00 3.01e-02  3e-02  3e-02 0:04.0
  300   4500 -1.587743043557102e+03 1.7e+00 4.41e-03  3e-03  5e-03 0:06.0
  400   6000 -1.587764374789383e+03 2.5e+00 1.58e-03  1e-03  2e-03 0:07.9
  500   7500 -1.587767434673144e+03 3.3e+00 5.23e-04  3e-04  1e-03 0:09.9
  600   9000 -1.587767758808977e+03 3.8e+00 1.66e-04  1e-04  3e-04 0:11.9
  700  10500 -1.587767837613337e+03 6.4e+00 7.71e-05  5e-05  3e-04 0:13.9
  800  12000 -1.587767841784837e+03 7.3e+00 7.95e-06  4e-06  3e-05 0:15.9
  900  13500 -1.587767841843046e+03 7.8e+00 1.82e-06  9e-07  7e-06 0:17.8
 1000  15000 -1.587767841845029e+03 8.2e+00 2.17e-07  1e-07  8e-07 0:19.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.587767841845053e+03 8.5e+00 2.96e-08  1e-08  1e-07 0:21.8
 1131  16965 -1.587767841845053e+03 8.7e+00 1.90e-08  9e-09  7e-08 0:22.4
termination on tolfun=1e-11 (Mon Apr 13 

...

  100   1500 -2.643355007114806e+02 1.4e+00 3.27e-01  3e-01  3e-01 0:02.0
  200   3000 -3.594832704646199e+02 1.6e+00 5.70e-02  5e-02  6e-02 0:04.0
  300   4500 -3.627895232714145e+02 1.9e+00 8.20e-03  6e-03  9e-03 0:05.9
  400   6000 -3.628483428320483e+02 2.0e+00 1.29e-03  9e-04  1e-03 0:07.9
  500   7500 -3.628492865220685e+02 2.2e+00 1.73e-04  1e-04  2e-04 0:09.9
  600   9000 -3.628493018977949e+02 2.5e+00 2.91e-05  2e-05  4e-05 0:11.9
  700  10500 -3.628493023302240e+02 2.8e+00 3.09e-06  2e-06  5e-06 0:13.8
  800  12000 -3.628493023347226e+02 3.1e+00 3.66e-07  2e-07  6e-07 0:15.8
  900  13500 -3.628493023347801e+02 3.3e+00 4.50e-08  2e-08  7e-08 0:17.8
  945  14175 -3.628493023347803e+02 3.4e+00 2.80e-08  1e-08  4e-08 0:18.7
termination on tolfun=1e-11 (Mon Apr 13 20:30:00 2020)
final/bestever f-value = -3.628493e+02 -3.628493e+02
incumbent solution: [ 4.25561989 -5.         -5.          5.         -4.65933916  5.
 -5.          5.         ...]
std deviations: [2.62279339e-08 1.653

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2221.58489789] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2204.40926331] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 8.455641743171439e+02 1.4e+00 3.50e-01  3e-01  4e-01 0:01.9
  200   3000 7.586053880723900e+02 1.5e+00 2.64e-02  2e-02  3e-02 0:03.8
  300   4500 7.580804036084653e+02 1.5e+00 1.42e-03  1e-03  1e-03 0:05.7
  400   6000 7.580793738532141e+02 1.5e+00 7.53e-05  6e-05  7e-05 0:07.6
  500   7500 7.580793712026232e+02 1.6e+00 4.22e-06  3e-06  4e-06 0:09.5
  600   9000 7.580793711919027e+02 1.6e+00 2.75e-07  2e-07  2e-07 0:11.4
  700  10500 7.580793711918741e+02 1.7e+00 2.61e-08  2e-08  2e-08 0:13.3
  728  10920 7.580793711918743e+02 1.7e+00 2.05e-08  1e-08  2e-08 0:13.8
termination on tolfun=1e-11 (Mon Apr 13 20:30:16 2020)
final/bestever f-value = 7.580794e+02 7.580794e+02
incumbent solution: [-0.582791    1.65000191  1.00086084 -0.70876561  1.02024488 -1.09544783
 -0.13443928 -0.8278747  ...]
std deviations: [1.60304348e-08 1.60469163e-08 1.58680323e-08 1.60840755e-08
 1.64490351e-08 1.52775784e-08 1.63092111e-08 1.68427952e-08 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimen

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2260.23872349] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=2260.238723, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=2260.238723, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -1.403174514763331e+03 1.5e+00 4.22e-01  4e-01  4e-01 0:02.0
  200   3000 -1.585347375056430e+03 1.5e+00 5.44e-02  5e-02  6e-02 0:04.0
  300   4500 -1.587701076411347e+03 1.7e+00 7.00e-03  6e-03  8e-03 0:06.0
  400   6000 -1.587762228289438e+03 2.6e+00 1.55e-03  1e-03  2e-03 0:08.0
  500   7500 -1.587766739736845e+03 3.8e+00 7.73e-04  5e-04  2e-03 0:10.0
  600   9000 -1.587767792245803e+03 6.0e+00 2.42e-04  2e-04  9e-04 0:11.9
  700  10500 -1.587767839843462e+03 6.8e+00 3.94e-05  2e-05  1e-04 0:14.0
  800  12000 -1.587767841742626e+03 7.3e+00 1.02e-05  6e-06  4e-05 0:16.0
  900  13500 -1.587767841843429e+03 7.7e+00 1.80e-06  1e-06  7e-06 0:17.9
 1000  15000 -1.587767841845036e+03 8.2e+00 1.99e-07  1e-07  7e-07 0:19.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.587767841845053e+03 8.3e+00 2.79e-08  1e-08  1e-07 0:21.9
 1121  16815 -1.587767841845053e+03 8.5e+00 2.11e-08  1e-08  8e-08 0:22.4
termination on tolfun=1e-11 (Mon Apr 13 

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F24_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F24(X_Values.iloc[i,:50])
    SVM_Fun [i] = F24(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F24(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([656.71411025]), count=array([1])) 9.873519156224141e-05
SVM
ModeResult(mode=array([1794.27610132]), count=array([1])) 296.9003558990098
ELN
ModeResult(mode=array([3011.65324623]), count=array([1])) 13.439824421563472
